### 1. prep

In [ ]:
// import spark
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.ArrayType
import org.apache.spark.sql.SaveMode

In [ ]:
// create spark session
val spark = SparkSession.builder().appName("medistream-05").getOrCreate()

In [ ]:
// get start time
val st = System.currentTimeMillis()

In [ ]:
// set paths
val savePath = savePath
val jsonPath = jsonPath

In [ ]:
// read data
val data = spark.read.json(jsonPath)

### 2. hospital key의 json object 전처리

In [ ]:
// hospital key의 json object를 hospital_data의 변수에 할당했다. alias("h") 옵션을 주어 이름을 "h"로 변경했다.
val hospitalData = data.select(explode(col("hospital")).alias("h"))

// hospital_data에서 필요한 item을 가져와서 hospital_df의 변수에 할당했다.
val hospitalDataSelected = hospitalData.select(
    col("h.id").alias("id"),
    col("h.name").alias("name"),
    col("h.category").alias("category"),
    col("h.category_code").alias("category_code"),
    col("h.category_code_list").alias("category_code_list"),
    col("h.category_count").alias("category_count"),
    col("h.description").alias("description"),
    col("h.road_address").alias("road_address"),
    col("h.road").alias("road"),
    col("h.rcode").alias("rcode"),
    col("h.virtual_phone").alias("virtual_phone"),
    col("h.phone").alias("phone"),
    col("h.payment_info").alias("payment_info"),
    col("h.conveniences").alias("conveniences"),
    col("h.review_setting.keyword").alias("review_keyword"),
    col("h.keywords").alias("keywords"),
    col("h.booking_business_id").alias("booking_business_id"),
    col("h.booking_display_name").alias("booking_display_name"),
    col("h.visitor_reviews_score").alias("visitor_reviews_score"),
    col("h.visitor_reviews_total").alias("visitor_reviews_total"),
    col("h.visitor_reviews_text_review_total").alias("visitor_reviews_text_review_total"),
    col("h.images").alias("images"),
    col("h.homepages.etc").alias("homepages_etc"),
    col("h.homepages.repr").alias("homepages_repr"),
    col("h.homepages.repr.url").alias("is_rep"), // isRep?
    col("h.booking_url").alias("booking_url"),
    col("h.talktalk_url").alias("talktalk_url"),
    col("h.coordinate.x").alias("lon"),
    col("h.coordinate.y").alias("lat")
    // 아래부터 hospitalDataSelected의 전처리 과정을 진행한다.
    // 아래부터 문자열 변경
).withColumn(
    "description", 
    regexp_replace(col("description"), "[\n\r*,]", "")
).withColumn(
    "road", 
    regexp_replace(col("road"), "[\n\r*,]", "")
).withColumn(
    "review_keyword", 
    regexp_replace(col("review_keyword"), "[\\\"]", "")
    // 아래부터 기타 전처리
).withColumn(
    // get description's length
    "description_length", length(col("description"))
).withColumn(
    // count images
    "images_count", size(col("images"))
).withColumn(
    // get photo_review_ratio
    "photo_review_ratio", (col("visitor_reviews_total") - col("visitor_reviews_text_review_total")) / col("visitor_reviews_total")
).withColumn(
    // get homepages' urls
    "homepages_url", 
    flatten(array(
        array(col("homepages_repr.url")), 
        col("homepages_etc.url")
    ))
).withColumn(
    // get homepages' types
    "homepages_type", 
    flatten(array(
        array(col("homepages_repr.type")), 
        col("homepages_etc.type")
    ))
).withColumn(
    // get homepages' order
    "homepages_order", 
    when(col("homepages_repr.order").isNull, array(lit(0)))
    .otherwise(
        flatten(array(
            array(col("homepages_repr.order")), 
            col("homepages_etc.order")
        ))
    )
).withColumn(
    // get boolean of smart phone
    "is_smart_phone", col("phone").startsWith("010")
).withColumn(
    // get boolean of zero pay
    "is_zero_pay", array_contains(col("payment_info"), "제로페이")
).withColumn(
    // get boolean of dead url
    "is_dead_url", 
    flatten(array(
        array(col("homepages_repr.isDeadUrl")), 
        col("homepages_etc.isDeadUrl")
    ))
).withColumn(
    // get 1st keyword
    "keywords_1", col("keywords")(0)
).withColumn(
    // get 2nd keyword
    "keywords_2", col("keywords")(1)
).withColumn(
    // get 3rd keyword
    "keywords_3", col("keywords")(2)
).withColumn(
    // get 4th keyword
    "keywords_4", col("keywords")(3)
).withColumn(
    // get 5th keyword
    "keywords_5", col("keywords")(4)
).drop(
    // drop unnecessary columns
    "images", 
    "keywords", 
    "homepages_repr", 
    "homepages_etc"
)


// 배열 삽입을 위한 전처리 과정

// get array type columns
val arrColList = hospitalDataSelected.schema.fields.filter(_.dataType.isInstanceOf[ArrayType]).map(_.name)

// get *var* hospitalDf
var hospitalDf = hospitalDataSelected

// concat_ws to array type columns
arrColList.foreach { arrCol =>
  hospitalDf = hospitalDf.withColumn(arrCol, concat_ws(",", col(arrCol)))
}

### 3. root key의 json object 전처리

In [ ]:
// root key의 json object를 불러온다. root key의 alias는 r, 변수명은 root_data이다.

// root key의 json object에서 필요한 item을 가져와서 root_df에 할당했다.
val rootData = data.select(explode(col("root")).alias("r"))

// root_df의 문자열 전처리를 진행했다.
val rootDf = rootData.select(
    regexp_extract(col("r.hospital.base.__ref"), "HospitalBase:([\\w]+)", 1).alias("root_id"),
    col("r.hospital.fsasReviews.total").alias("fsas_reviews_count"),
    col("r.hospital.kinQna.answerCount").alias("kin_qna_count")
)

### 4. join dataframes

In [ ]:
// hospital_df와 root_df를 left outer join해서 df로 만들었다. 
// 올바른 값이 가져오기 위해 id를 비교했다. 
// 불필요해진 root_id 값은 drop한다.
val df = hospitalDf.join(rootDf, hospitalDf("id") === rootDf("root_id"), "left_outer").drop("root_id")

### 5. save dataframe

In [ ]:
// save df
df.dropDuplicates().write.mode("overwrite").parquet(savePath)

### 6. get task time

In [ ]:
// calculate task time
val ft = System.currentTimeMillis()
println(s"Spark task time: ${(ft - st)/1000} s")

### 7. upload to redshift

In [ ]:
// set envs
val jdbcUrl = "<jdbc_url>"
val tempDir = "<temp_dir>"
val dbTable = "<db_table>"

// save
df.write
  .format("io.github.spark_redshift_community.spark.redshift")
  .option("driver", "com.amazon.redshift.jdbc42.Driver")
  .option("forward_spark_s3_credentials", "true")
  .option("url", jdbcUrl)
  .option("dbtable", dbTable)
  .option("tempdir", tempDir)
  .mode(SaveMode.Overwrite)
  .save()

In [ ]:
// stop spark session
spark.stop()